<a href="https://colab.research.google.com/github/hyulianton/BigData/blob/main/Klasifikasi_dengan_H2O_ai_(AutoML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Contoh 2: Klasifikasi dengan H2O.ai (AutoML)

**Tujuan:** Mendemonstrasikan bagaimana melakukan klasifikasi menggunakan H2O.ai, khususnya fitur AutoML-nya, pada dataset Wine.

**Dataset:** Wine Dataset (dataset klasik untuk klasifikasi, tersedia di scikit-learn).

**Prasyarat:**

-   H2O Python _client_ terinstal (`pip install h2o`).

In [2]:
!pip install h2o

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 4.6 MB/s eta 0:00:00


In [9]:
# --- Langkah 1: Inisialisasi H2O Cluster ---
import h2o
h2o.init(nthreads=-1, max_mem_size="4G")

print("H2O cluster berhasil diinisialisasi.")
# URL H2O Flow akan otomatis ditampilkan oleh h2o.init() di output konsol.

# --- Langkah 2: Muat Dataset (Simulasi Data Big Data) ---
from sklearn.datasets import load_wine
import pandas as pd

wine = load_wine()
wine_df_pandas = pd.DataFrame(data=wine.data, columns=wine.feature_names)
wine_df_pandas['target'] = wine.target

print("\nDataset Wine (Pandas DataFrame):")
print(wine_df_pandas.head())
print(f"Ukuran dataset: {wine_df_pandas.shape[0]} baris, {wine_df_pandas.shape[1]} kolom.")

wine_df_h2o = h2o.H2OFrame(wine_df_pandas)

print("\nDataset Wine (H2OFrame - Info):")
wine_df_h2o.summary()
print("\nDataset Wine (H2OFrame - Contoh Data):")
wine_df_h2o.head(5)

wine_df_h2o['target'] = wine_df_h2o['target'].asfactor()
print(f"Tipe kolom 'target' setelah konversi: {wine_df_h2o['target'].type}")

# --- Langkah 3: Pembagian Data (Training dan Testing) ---
train_h2o, test_h2o = wine_df_h2o.split_frame(ratios=[0.7], seed=42)

print(f"\nUkuran data training (H2OFrame): {train_h2o.nrows} baris.")
print(f"Ukuran data testing (H2OFrame): {test_h2o.nrows} baris.")

# --- Langkah 4: Definisikan Kolom Fitur dan Kolom Target ---
x = wine_df_h2o.columns
y = 'target'
x.remove(y)

# --- Langkah 5: Menggunakan H2O AutoML ---
from h2o.automl import H2OAutoML

print("\nMemulai proses H2O AutoML...")

aml = H2OAutoML(max_models=10, seed=42) # Tanpa exclude_algos

aml.train(x=x, y=y, training_frame=train_h2o, leaderboard_frame=test_h2o)

print("Proses AutoML selesai.")

# --- Langkah 6: Melihat Hasil AutoML (Leaderboard) ---
print("\nAutoML Leaderboard (Model Terbaik):")
print(aml.leaderboard)

best_model = aml.leader

print(f"\nModel terbaik dari AutoML: {best_model.model_id}")

# --- Langkah 7: Evaluasi Model Terbaik ---
performance = best_model.model_performance(test_h2o)

print("\nPerforma Model Terbaik pada data testing:")

# Dapatkan confusion matrix sebagai Pandas DataFrame
conf_matrix = performance.confusion_matrix().as_data_frame()
print(f"Confusion Matrix:\n{conf_matrix}")

# --- Perbaikan di sini: Menghitung Akurasi dengan benar ---

# Hapus kolom 'Error' dan 'Rate' yang non-numerik
# Juga, H2O menyertakan baris total di akhir, kita hanya butuh baris kelas aktual.
# Biasanya, jumlah baris untuk kelas adalah sama dengan jumlah kolom kelas.
# Untuk dataset Wine, ada 3 kelas (0, 1, 2).
num_classes = len(conf_matrix.columns) - 2 # Kurangi 2 untuk kolom 'Error' dan 'Rate'

# Ambil hanya bagian numerik dari confusion matrix, tanpa baris total dan kolom non-numerik
# Kita hanya perlu 3 baris pertama (untuk kelas 0, 1, 2) dan 3 kolom pertama (untuk prediksi kelas 0, 1, 2).
# Asumsi kolom kelas diberi nama '0', '1', '2'
numeric_data = conf_matrix.iloc[:num_classes, :num_classes]

# Hitung jumlah prediksi benar (elemen diagonal)
correct_predictions = numeric_data.values.trace() # trace() menjumlahkan elemen diagonal matrix numpy

# Hitung total prediksi (jumlah semua elemen di matriks kelas aktual vs. prediksi)
total_predictions = numeric_data.values.sum()

# Pastikan total_predictions tidak nol untuk menghindari ZeroDivisionError
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"Accuracy (dihitung dari Confusion Matrix): {accuracy:.4f}")

# H2O juga menyediakan metrik lain yang umum untuk multi-kelas
print(f"Logloss: {performance.logloss():.4f}")
print(f"Mean Per Class Error: {performance.mean_per_class_error():.4f}")
# Anda juga bisa coba mengakses akurasi melalui performance.accuracy_multiclass(),
# namun bergantung pada versi H2O dan bagaimana metriknya diekspos.
# Jika ingin, bisa coba:
# try:
#     print(f"Accuracy (dari H2O method): {performance.accuracy_multiclass():.4f}")
# except AttributeError:
#     pass # Abaikan jika method tidak ada


# --- Langkah 8: Menghentikan H2O Cluster ---
h2o.cluster().shutdown(prompt=False)
print("\nH2O cluster dihentikan.")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.27" 2025-04-15; OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpe7mctdnm
  JVM stdout: /tmp/tmpe7mctdnm/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpe7mctdnm/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,1 month and 29 days
H2O_cluster_name:,H2O_from_python_unknownUser_168uk2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


H2O cluster berhasil diinisialisasi.

Dataset Wine (Pandas DataFrame):
   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  

<ipython-input-9-8823febd19dd>:23: H2ODeprecationWarning: ``summary()`` is deprecated, please use ``show_summary()`` or ``get_summary()`` instead
  wine_df_h2o.summary()


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
type,real,real,real,real,int,real,real,real,real,real,real,real,int,int
mins,11.03,0.74,1.36,10.6,70.0,0.98,0.34,0.13,0.41,1.28,0.48,1.27,278.0,0.0
mean,13.000617977528085,2.336348314606743,2.366516853932584,19.494943820224723,99.74157303370781,2.295112359550562,2.0292696629213474,0.36185393258426946,1.5908988764044947,5.058089882022471,0.9574494382022474,2.6116853932584254,746.8932584269659,0.9382022471910113
maxs,14.83,5.8,3.23,30.0,162.0,3.88,5.08,0.66,3.58,13.0,1.71,4.0,1680.0,2.0
sigma,0.811826538005858,1.1171460976144625,0.27434400906081485,3.339563767173504,14.282483515295652,0.6258510488339892,0.9988586850169471,0.12445334029667941,0.5723588626747612,2.318285871822413,0.22857156582982327,0.7099904287650503,314.9074742768492,0.7750349899850563
zeros,0,0,0,0,0,0,0,0,0,0,0,0,0,59
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185.0,0.0



Dataset Wine (H2OFrame - Contoh Data):
Tipe kolom 'target' setelah konversi: <bound method H2OFrame.type of H2OFrame({'_ex': <Expr(cols_py <Expr(:= <Expr()#Key_Frame__upload_bbe0e207b587cb12d574118165564af8.hex> <Expr(as.factor <Expr(cols_py <Expr()#Key_Frame__upload_bbe0e207b587cb12d574118165564af8.hex> 'target'); scalar>); scalar> 13 None); scalar> 'target'); scalar>})>

Ukuran data training (H2OFrame): 129 baris.
Ukuran data testing (H2OFrame): 49 baris.

Memulai proses H2O AutoML...
AutoML progress: |█████
06:21:42.647: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 129.0.

██████████████████████████████████████████████████████████| (done) 100%
Proses AutoML selesai.

AutoML Leaderboard (Model Terbaik):
model_id                                                  mean_per_class_error     logloss       rmse          mse
XRT_1_AutoML_1_20250527_62127                                                0  0.

Senang melihat program H2O Anda akhirnya berjalan dengan baik! Mari kita bedah arti dari setiap bagian *output* tersebut.

---

## Membedah Output AutoML H2O Anda

### 1. `Memulai proses H2O AutoML...`
`AutoML progress: |█████`
`06:21:42.647: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 129.0.`

Ini adalah pesan awal dari proses AutoML:

* **`Memulai proses H2O AutoML...`**: H2O sedang menginisiasi pencarian model terbaik secara otomatis.
* **`AutoML progress: |█████`**: Ini adalah *progress bar* visual yang menunjukkan seberapa jauh proses AutoML berjalan. Anda akan melihatnya mengisi hingga 100%.
* **`_min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 129.0.`**: Ini adalah **peringatan** dari H2O, bukan *error* kritis.
    * H2O memiliki parameter internal seperti `_min_rows` yang digunakan untuk validasi silang (cross-validation) atau teknik *sampling* tertentu. Parameter ini secara *default* mungkin mengharapkan ukuran *dataset* pelatihan minimum (misalnya 200 baris) untuk melakukan pembagian data internalnya dengan efektif.
    * *Dataset* pelatihan Anda hanya memiliki 129 baris (`train_h2o.nrows`). Karena jumlah baris ini di bawah ambang batas *default* (200 baris), H2O memberitahukan bahwa *dataset* Anda terlalu kecil untuk teknik *split* tertentu yang mungkin ingin digunakan oleh AutoML.
    * **Tidak perlu khawatir**, ini hanya peringatan dan proses AutoML tetap akan berjalan. Untuk *dataset* sekecil Wine, peringatan ini sangat umum. Dalam Big Data yang sesungguhnya, Anda tidak akan melihat peringatan ini karena jumlah barisnya akan jauh di atas ambang batas.

---

### 2. `██████████████████████████████████████████████████████████| (done) 100%`
`Proses AutoML selesai.`

* Ini menunjukkan bahwa **proses AutoML telah berhasil diselesaikan**. H2O telah mencoba berbagai algoritma, menyetelnya, dan membangun *leaderboard* model.

---

### 3. `AutoML Leaderboard (Model Terbaik):`

| model_id | mean_per_class_error | logloss | rmse | mse |
| :------------------------------------------------------ | :------------------- | :---------- | :--------- | :---------- |
| `XRT_1_AutoML_1_20250527_62127` | `0` | `0.168363` | `0.197201` | `0.0388882` |
| `GBM_4_AutoML_1_20250527_62127` | `0` | `0.0168734` | `0.0527453` | `0.00278206` |
| `StackedEnsemble_AllModels_1_AutoML_1_20250527_62127` | `0` | `0.0396081` | `0.0812332` | `0.00659884` |
| `XGBoost_1_AutoML_1_20250527_62127` | `0` | `0.281922` | `0.262702` | `0.0690124` |
| `StackedEnsemble_BestOfFamily_1_AutoML_1_20250527_62127` | `0` | `0.0453559` | `0.0902253` | `0.00814061` |
| `GBM_2_AutoML_1_20250527_62127` | `0` | `0.0131054` | `0.0418693` | `0.00175304` |
| `GLM_1_AutoML_1_20250527_62127` | `0` | `0.0774136` | `0.13132` | `0.017245` |
| `DRF_1_AutoML_1_20250527_62127` | `0` | `0.181723` | `0.206043` | `0.0424536` |
| `GBM_3_AutoML_1_20250527_62127` | `0` | `0.0043451` | `0.0147223` | `0.000216746` |
| `GBM_5_AutoML_1_20250527_62127` | `0` | `0.00410362` | `0.0110026` | `0.000121058` |
| `[12 rows x 5 columns]` | | | | |

Ini adalah **Leaderboard** yang menampilkan semua model yang dilatih oleh AutoML, diurutkan berdasarkan metrik evaluasi terbaik (biasanya `mean_per_class_error` atau `logloss` untuk klasifikasi).

* **`model_id`**: Nama unik untuk setiap model yang dilatih.
    * `XRT`: eXtreme Random Trees.
    * `GBM`: Gradient Boosting Machine.
    * `StackedEnsemble`: Model yang menggabungkan prediksi dari beberapa model lain.
    * `XGBoost`: eXtreme Gradient Boosting.
    * `GLM`: Generalized Linear Model.
    * `DRF`: Distributed Random Forest.
* **`mean_per_class_error`**: Ini adalah metrik utama untuk klasifikasi multi-kelas. Nilai `0` untuk semua model di sini berarti bahwa **setiap model yang dilatih di *leaderboard* ini mencapai akurasi 100% pada *test set*** Anda untuk setiap kelasnya. Ini sangat luar biasa!
* **`logloss`**: Logarithmic Loss. Nilai yang lebih rendah menunjukkan probabilitas prediksi yang lebih baik. Semakin mendekati nol, semakin sempurna model memprediksi probabilitas.
* **`rmse`**: Root Mean Squared Error. Ini adalah metrik yang lebih umum untuk regresi, tetapi H2O mungkin juga menghitungnya untuk klasifikasi sebagai indikator umum seberapa jauh prediksi numerik dari nilai sebenarnya. Nilai yang lebih rendah lebih baik.
* **`mse`**: Mean Squared Error. Mirip dengan RMSE, yaitu kuadrat dari RMSE.

**Urutan Leaderboard**: Model diurutkan dari yang terbaik ke yang terburuk berdasarkan metrik utama. Dalam kasus Anda, karena semua model memiliki `mean_per_class_error` yang sama (yaitu 0), maka H2O akan menggunakan `logloss` sebagai metrik sekunder untuk mengurutkan (nilai `logloss` terkecil ada di bagian atas).

---

### 4. `Model terbaik dari AutoML: XRT_1_AutoML_1_20250527_62127`

* Ini menunjukkan bahwa **model dengan kinerja terbaik** di antara semua model yang dicoba oleh AutoML adalah **XRT (eXtreme Random Trees)** dengan ID tersebut. H2O secara otomatis memilih model ini sebagai `aml.leader` karena ia memiliki `mean_per_class_error` terendah (dalam hal ini, 0) dan `logloss` terbaik di antara semua model yang memiliki `mean_per_class_error` 0.

---

### 5. `Performa Model Terbaik pada data testing:`
`Confusion Matrix:`
`      0     1     2  Error    Rate`
`0  16.0   0.0   0.0    0.0  0 / 16`
`1   0.0  19.0   0.0    0.0  0 / 19`
`2   0.0   0.0  14.0    0.0  0 / 14`
`3  16.0  19.0  14.0    0.0  0 / 49`
`Accuracy (dihitung dari Confusion Matrix): 1.0000`
`Logloss: 0.1684`
`Mean Per Class Error: 0.0000`

Bagian ini merinci kinerja model `XRT_1_AutoML_1_20250527_62127` (model terbaik) pada *test set* Anda:

* **Confusion Matrix**: Matriks ini menunjukkan bahwa model `XRT` ini berhasil memprediksi **semua sampel dengan benar** pada *test set*.
    * 16 sampel aktual kelas 0 diprediksi sebagai kelas 0.
    * 19 sampel aktual kelas 1 diprediksi sebagai kelas 1.
    * 14 sampel aktual kelas 2 diprediksi sebagai kelas 2.
    * Tidak ada kesalahan prediksi (semua nilai di luar diagonal adalah 0).
* **Accuracy (dihitung dari Confusion Matrix): 1.0000**: Ini mengkonfirmasi bahwa model Anda memiliki **akurasi 100%** pada *test set*. Ini adalah hasil yang sangat luar biasa dan menunjukkan bahwa *dataset* Wine Anda sangat mudah dipisahkan atau modelnya sangat efektif.
* **Logloss: 0.1684**: Meskipun akurasi 100%, Logloss-nya tidak nol sempurna. Ini berarti model masih memiliki sedikit ketidakpastian dalam probabilitas prediksinya, tetapi cukup yakin untuk menghasilkan prediksi kelas yang benar secara mutlak.
* **Mean Per Class Error: 0.0000**: Mengkonfirmasi tidak ada kesalahan rata-rata per kelas.

---

### 6. `H2O session _sid_b278 closed.`
`H2O cluster dihentikan.`

* Ini adalah konfirmasi penutupan sesi dan *cluster* H2O, seperti yang sudah dijelaskan sebelumnya.

---

**Kesimpulan Umum:**

Output Anda menunjukkan bahwa Anda telah berhasil melatih model klasifikasi menggunakan H2O AutoML pada *dataset* Wine, dan model terbaik yang ditemukan (XRT) mencapai **akurasi 100%** pada *test set*. Ini adalah hasil yang fantastis untuk *dataset* ini!

Peringatan `_min_rows param` di awal adalah hal yang normal untuk *dataset* kecil dan tidak menghalangi proses AutoML.

Apakah ada bagian lain dari *output* yang ingin Anda tanyakan lebih lanjut?